In [9]:
import pandas as pd
import psycopg2

# Reading "Sales" and "MetaClean" data from a CSV file into a Pandas DataFrame
sales_data = pd.read_csv(r'c:\Users\leath\Documents\AUAS\3. Digital Business Fundamentals\Sales Cleaned.csv')
metaclean_data = pd.read_csv(r"c:\Users\leath\Documents\AUAS\3. Digital Business Fundamentals\MetaClean Cleaned.csv")

# Establishing a connection to PostgreSQL database
conn = psycopg2.connect(
    dbname = "Movie Industry Database",
    user = "postgres",
    password = "Quickborn123",
    host = "localhost",
    port = "5432"
)

# Creating a cursor to execute SQL commands 
cursor = conn.cursor()

# Creating an empty dictionary to map genre names to genre IDs
genre_id_mapping = {}

# Creating a loop to go through the metaclean dataframe
for _, genre_row in metaclean_data.iterrows():
    # Splitting the 'genre' column value into a list of genre names
    genre_names = genre_row['genre'].split(',') 
    # Getting the 'movie_title' from the current row
    movie_title = genre_row['movie_title']  

    # Execute a SQL query to retrieve the 'movie_id' for a given 'movie_title'
    cursor.execute("SELECT movie_id FROM movies WHERE movie_title = %s", (movie_title,))
    movie_id_result = cursor.fetchone()

    if movie_id_result is not None:
        # Extract the 'movie_id' from the query result
        movie_id = movie_id_result[0]

        # Loop through each genre name associated with the movie
        for genre_name in genre_names:
            genre_name = genre_name.strip()  # Removing leading/trailing spaces

            # Checking if the genre name is already in the mapping dictionary. 
            if genre_name in genre_id_mapping:
                genre_id = genre_id_mapping[genre_name]
            else:
                # Inserting the genre into the genres table and return the generated genre ID
                cursor.execute("INSERT INTO genres (genre_name) VALUES (%s) RETURNING genre_id", (genre_name,))
                genre_id = cursor.fetchone()[0]
                genre_id_mapping[genre_name] = genre_id 

            # Check if a record already exists in the 'moviegenrelink' table for the movie and genre
            cursor.execute("SELECT 1 FROM moviegenrelink WHERE movie_id = %s AND genre_id = %s", (movie_id, genre_id))
            existing_record = cursor.fetchone()

              # If no record exists, inserting a new record into the moviegenrelink table
            if not existing_record:
                cursor.execute("INSERT INTO moviegenrelink (movie_id, genre_id) VALUES (%s, %s)", (movie_id, genre_id))
                conn.commit()

# Closing cursor and database connection
cursor.close()
conn.close()
